# Réseau neuronal convolutif

## Imports

In [3]:
import numpy as np 
import shutil # pour les dossiers
import pandas as pd
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time
import os
import copy

## Chargement des images

In [8]:
def metrique(path):
    files = os.listdir(path);
    contenu = [];
    for file in files:
        if file.endswith('.json'):
            openFile = open(path+file, "r");
            contenu.append(json.loads(openFile.read())["Image"]);
            openFile.close();
    return contenu;

In [9]:
chemin = "./train/";
contenu = [];
if not os.path.isdir(chemin):
    print('Rajouter le dossier train dans le dossier courant :) ! ');
else :
    contenu = metrique(chemin);
df = pd.DataFrame(contenu)
# on laisse Species car c'est égale au nombre de classe
df.drop(["Genus","ClassId","Family","Vote","Location","Latitude","Longitude","Date","Author","Content","MediaId", "LearnTag", "ImageId2014","ObservationId2014","YearInCLEF","ObservationId"], axis='columns', inplace=True)
df = df.drop_duplicates()
classe  = df.to_numpy();
print(len(classe))


50


## On récupère le nom des png ainsi que leur classe

In [18]:
X = [];
Y = [];
files = os.listdir("./train")
for file in files:
    if file.endswith('.jpg'):
        X.append(file);
        nameFile = file.split('.')[0];
        for file_class in files:
            nameFileAutre = file_class.split('.')[0];
            if nameFile==nameFileAutre and file_class.endswith('.json'):
                fichierSrc = open("./train/"+file_class, "r");
                contenu = fichierSrc.read();
                monJson = json.loads(contenu)
                Y.append(monJson["Image"]["Species"])
                break;


In [11]:
# on a bien une classe par image !
if(len(X) == len(Y)):
    print("OK")
else:
    print("KO")

OK


In [22]:
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42, stratify=Y)

# 60% train 20% test 20% val 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42,stratify=Y)
X_train, X_val, y_train, y_val = train_test_split( X_train, y_train, test_size=0.25, random_state=42,stratify=y_train)

## Mise en place de l'architecture dossier pour avoir les image_datasets

In [8]:
# Création des répertoires ! 
directory = "CNN"
for classe in y_train : 
    if(not os.path.isdir('./CNN/train/'+classe)):
        os.mkdir('./CNN/train/'+classe)
        
for classe in y_test : 
    if(not os.path.isdir('./CNN/val/'+classe)):
        os.mkdir('./CNN/val/'+classe)
        
# on met les images dans les répertoires precédents

# on supprime le contenu

directorys=os.listdir('./CNN/train/')
for i in range(0,len(directory)):
    files=os.listdir('./CNN/train/'+directorys[i])
    for j in range(0,len(files)):
        os.remove('./CNN/train/'+directorys[i]+'/'+files[j])

directorys=os.listdir('./CNN/val/')
for i in range(0,len(directory)):
    files=os.listdir('./CNN/val/'+directorys[i])
    for j in range(0,len(files)):
        os.remove('./CNN/val/'+directorys[i]+'/'+files[j])

# on met le contenu

for i in range(len(X_train)):
    shutil.copyfile('./train/'+X_train[i],'./CNN/train/'+y_train[i]+'/'+X_train[i])
    
for i in range(len(X_test)):
    shutil.copyfile('./train/'+X_test[i],'./CNN/val/'+y_test[i]+'/'+X_test[i])
    

## Mise en place du CNN

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_dir = 'CNN'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
classe_len = len(image_datasets['train'].classes)
model_ft = models.resnet18(pretrained=True)
print(device)


cuda:0


In [25]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    evolution_accuracy = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            evolution_accuracy.append(epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print("Evolution de l'accuracy pour savoir si on doit augmenter l'épochs : ")
    evolution_accuracy.plot(kind='line', sharex=False, title='Evolution accuracy par epoch')
    plt.show()

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [13]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [13]:
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
class_names = image_datasets['train'].classes
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [14]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=15)

Epoch 0/14
----------
train Loss: 2.7724 Acc: 0.2780
val Loss: 1.2202 Acc: 0.6312

Epoch 1/14
----------
train Loss: 1.6911 Acc: 0.5243
val Loss: 0.8781 Acc: 0.7437

Epoch 2/14
----------
train Loss: 1.3339 Acc: 0.6171
val Loss: 0.8839 Acc: 0.7341

Epoch 3/14
----------
train Loss: 1.1863 Acc: 0.6794
val Loss: 0.7560 Acc: 0.7995

Epoch 4/14
----------
train Loss: 1.0488 Acc: 0.6983
val Loss: 0.6942 Acc: 0.8056

Epoch 5/14
----------
train Loss: 1.0006 Acc: 0.7121
val Loss: 0.7154 Acc: 0.8091

Epoch 6/14
----------
train Loss: 0.8976 Acc: 0.7430
val Loss: 0.6420 Acc: 0.8317

Epoch 7/14
----------
train Loss: 0.6687 Acc: 0.8144
val Loss: 0.4728 Acc: 0.8657

Epoch 8/14
----------
train Loss: 0.5971 Acc: 0.8376
val Loss: 0.4630 Acc: 0.8657

Epoch 9/14
----------
train Loss: 0.5911 Acc: 0.8341
val Loss: 0.4598 Acc: 0.8727

Epoch 10/14
----------
train Loss: 0.5376 Acc: 0.8552
val Loss: 0.4436 Acc: 0.8692

Epoch 11/14
----------
train Loss: 0.5153 Acc: 0.8573
val Loss: 0.4282 Acc: 0.8779

Ep

## Autre test sans paramétrage du réseaux par ImageNet

In [15]:
model_ft = models.resnet18(pretrained=False)

In [16]:
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
class_names = image_datasets['train'].classes
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [17]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=15)

Epoch 0/14
----------
train Loss: 3.7817 Acc: 0.0735
val Loss: 3.4936 Acc: 0.1351

Epoch 1/14
----------
train Loss: 3.4286 Acc: 0.1173
val Loss: 3.2983 Acc: 0.1813

Epoch 2/14
----------
train Loss: 3.1656 Acc: 0.1715
val Loss: 3.6079 Acc: 0.2075

Epoch 3/14
----------
train Loss: 3.0432 Acc: 0.1856
val Loss: 3.8186 Acc: 0.2345

Epoch 4/14
----------
train Loss: 2.9014 Acc: 0.2278
val Loss: 2.6012 Acc: 0.3304

Epoch 5/14
----------
train Loss: 2.8143 Acc: 0.2376
val Loss: 2.9586 Acc: 0.2982

Epoch 6/14
----------
train Loss: 2.7404 Acc: 0.2467
val Loss: 2.5822 Acc: 0.3357

Epoch 7/14
----------
train Loss: 2.4386 Acc: 0.3287
val Loss: 2.3126 Acc: 0.3888

Epoch 8/14
----------
train Loss: 2.3346 Acc: 0.3545
val Loss: 2.1811 Acc: 0.4281

Epoch 9/14
----------
train Loss: 2.3333 Acc: 0.3644
val Loss: 2.2888 Acc: 0.4124

Epoch 10/14
----------
train Loss: 2.3283 Acc: 0.3520
val Loss: 2.2638 Acc: 0.4045

Epoch 11/14
----------
train Loss: 2.2770 Acc: 0.3687
val Loss: 2.1606 Acc: 0.4263

Ep

## Test en fixant le réseaux sauf pour la dernière couche  

In [23]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
class_names = image_datasets['train'].classes
model_conv.fc = nn.Linear(num_ftrs, len(class_names))

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [24]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=15)

Epoch 0/14
----------
train Loss: 3.4065 Acc: 0.1693
val Loss: 2.0665 Acc: 0.4237

Epoch 1/14
----------
train Loss: 2.4055 Acc: 0.3954
val Loss: 1.4438 Acc: 0.6059

Epoch 2/14
----------
train Loss: 1.9468 Acc: 0.4994
val Loss: 1.1678 Acc: 0.6661

Epoch 3/14
----------
train Loss: 1.7314 Acc: 0.5505
val Loss: 0.9100 Acc: 0.7341

Epoch 4/14
----------
train Loss: 1.5804 Acc: 0.5853
val Loss: 1.0032 Acc: 0.6966

Epoch 5/14
----------
train Loss: 1.5110 Acc: 0.5965
val Loss: 0.9275 Acc: 0.7280

Epoch 6/14
----------
train Loss: 1.4304 Acc: 0.6227
val Loss: 0.8335 Acc: 0.7498

Epoch 7/14
----------
train Loss: 1.2677 Acc: 0.6618
val Loss: 0.7874 Acc: 0.7629

Epoch 8/14
----------
train Loss: 1.2219 Acc: 0.6850
val Loss: 0.7746 Acc: 0.7672

Epoch 9/14
----------
train Loss: 1.2166 Acc: 0.6914
val Loss: 0.7829 Acc: 0.7620

Epoch 10/14
----------
train Loss: 1.2009 Acc: 0.6932
val Loss: 0.7671 Acc: 0.7742

Epoch 11/14
----------
train Loss: 1.1968 Acc: 0.7018
val Loss: 0.7805 Acc: 0.7707

Ep